In [171]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import Image, display
import plotly.io as pio
from scipy.stats import t

# 1.2.1 BTC Price and Dynamic DCA Allocation under Non-overlapping 1-year Rolling Windows (2018–2025)

In [ ]:
def plot_weight_price_by_year(
    start_year: int,
    end_year: int | None = None,
    file_path: str = "data/weight_price.csv"
):
    # Load data
    df = pd.read_csv(file_path)
    df["date"] = pd.to_datetime(df["date"])
    df = df.sort_values("date")

    # If only one year is given
    if end_year is None:
        end_year = start_year + 1

    # Window: June 1 -> May 31
    start_date = pd.Timestamp(f"{start_year}-06-01")
    end_date   = pd.Timestamp(f"{end_year}-05-31")

    # Filter by window
    df = df[(df["date"] >= start_date) & (df["date"] <= end_date)]

    # Plot
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # BTC Price (left axis)
    fig.add_trace(
        go.Scatter(
            x=df["date"],
            y=df["PriceUSD"],
            mode="lines",
            name="BTC Price"
        ),
        secondary_y=False
    )

    # Weight (right axis)
    fig.add_trace(
        go.Bar(
            x=df["date"],
            y=df["weight"],
            name="Weight",
            marker=dict(color="red", opacity=1)
        ),
        secondary_y=True
    )

    # X-axis: every June 1st
    fig.update_layout(
        title=f"BTC Price and Allocation Weight ({start_year} → {end_year})",
        xaxis=dict(
            tickmode="linear",
            dtick="M12",
            tick0=f"{start_year}-06-01"
        ),
        barmode="overlay"
    )

    fig.update_yaxes(title_text="BTC Price (USD)", secondary_y=False)
    fig.update_yaxes(title_text="Weight", secondary_y=True)

    fig.show()

    # Convert the figure to a PNG and render it as notebook output (so GitHub can display it)
    png_bytes = pio.to_image(
        fig,
        format="png",
        width=1400,    # Image width in pixels (increase for wider output)
        height=600,    # Image height in pixels
        scale=2         # Resolution multiplier (the most important for clarity)
    )

    # Display the PNG image inside the notebook
    display(Image(png_bytes))

# 2.1.1 SPD Percentile vs Allocation Weight (Aggregated Across Non-Overlapping 365-Day Windows: 2011–2025)

In [ ]:
def plot_percentile_vs_weight(
    file_path: str = "data/weight_price_ma20_spd_pct.csv",
    figsize: tuple = (6, 4),
    jitter_std: float = 0.3,
    alpha: float = 0.25
):
    # Load data
    df = pd.read_csv(file_path)

    # Convert to numeric and clean
    df["spd_percentile"] = pd.to_numeric(df["spd_percentile"], errors="coerce")
    df["weight"] = pd.to_numeric(df["weight"], errors="coerce")
    df = df.dropna(subset=["spd_percentile", "weight"])

    # Add jitter on x-axis
    jitter = np.random.normal(0, jitter_std, size=len(df))

    plt.figure(figsize=figsize)
    plt.scatter(
        df["spd_percentile"] + jitter,
        df["weight"],
        alpha=alpha
    )
    plt.xlabel("SPD Percentile")
    plt.ylabel("Weight")
    plt.title("SPD Percentile vs Weight (2011–2025)")
    plt.tight_layout()
    plt.show()

In [ ]:
def plot_weight_by_spd_bucket(
    file_path: str = "data/weight_price_ma20_spd_pct.csv",
    bucket_size: int = 20
):
    """
    Plot a boxplot of weight distribution by SPD percentile buckets,
    and highlight the mean value with a red dot (log scale).
    """

    # Load data
    df = pd.read_csv(file_path)

    # Convert columns to numeric and remove NaN rows
    df["spd_percentile"] = pd.to_numeric(df["spd_percentile"], errors="coerce")
    df["weight"] = pd.to_numeric(df["weight"], errors="coerce")
    df = df.dropna(subset=["spd_percentile", "weight"])

    # Create percentile buckets (e.g. 0–20, 20–40, ...)
    bins = list(range(0, 100 + bucket_size, bucket_size))
    df["bucket"] = pd.cut(df["spd_percentile"], bins=bins)

    # Collect values for each bucket
    bucket_groups = []
    labels = []

    for name, group in df.groupby("bucket", observed=True):
        bucket_groups.append(group["weight"].values)
        labels.append(str(name))

    # Plot
    plt.figure(figsize=(8, 4))

    bp = plt.boxplot(
        bucket_groups,
        labels=labels,
        showfliers=True,
        showmeans=True,
        meanline=False
    )

    # Use log scale for better visibility
    plt.yscale("log")

    # Force the mean points to be red and more visible
    for m in bp["means"]:
        m.set_marker("o")
        m.set_markerfacecolor("red")
        m.set_markeredgecolor("red")
        m.set_markersize(7)

    # Add custom legend for the mean
    plt.scatter([], [], color="red", label="Mean Weight")
    plt.legend()

    plt.xticks(rotation=45)
    plt.xlabel("SPD Percentile Bucket")
    plt.ylabel("Weight (log scale)")
    plt.title("Weight Distribution by SPD Percentile Bucket (Mean Highlighted)")
    plt.tight_layout()
    plt.show()

# 2.2 Simple Returns Win Rate in a yearly window: Dynamic vs Uniform DCA

In [ ]:
def dynamic_vs_uniform_return_by_window(
    file_path: str = "data/port_dca.csv",
    date_col: str = "date",
    dyn_col: str = "dyn_portfolio_value",
    uni_col: str = "uni_portfolio_value",
    budget: float = 1.0,
) -> tuple[float, pd.DataFrame]:
    """
    Assumptions
    ----------
    - Data is already concatenated by rolling windows in order
      (e.g. 2018-06-01~2019-06-01, then 2018-06-02~2019-06-02, ...)
    - A new window starts when date[t] <= date[t-1]
    - Each window uses the SAME budget, and budget is fully invested

    Return
    ------
    win_rate, result_df

    result_df contains:
    - window_id
    - start_date
    - end_date
    - dyn_return
    - uni_return
    """

    df = pd.read_csv(file_path)
    df[date_col] = pd.to_datetime(df[date_col])

    # identify window boundaries
    is_new_window = df[date_col].diff().le(pd.Timedelta(0))
    is_new_window.iloc[0] = True
    df["window_id"] = is_new_window.cumsum()

    results = []

    for wid, g in df.groupby("window_id", sort=False):

        g = g.sort_values(date_col)

        start_date = g[date_col].iloc[0]
        end_date = g[date_col].iloc[-1]

        final_dyn_value = g[dyn_col].iloc[-1]
        final_uni_value = g[uni_col].iloc[-1]

        if pd.isna(final_dyn_value) or pd.isna(final_uni_value):
            continue

        dyn_return = (final_dyn_value - budget) / budget
        uni_return = (final_uni_value - budget) / budget

        results.append(
            {
                "window_id": wid,
                "start_date": start_date,
                "end_date": end_date,
                "dyn_return": dyn_return,
                "uni_return": uni_return,
                "dyn_win_flag": int(dyn_return > uni_return),
            }
        )

    result_df = pd.DataFrame(results)

    if len(result_df) == 0:
        win_rate = np.nan
    else:
        win_rate = pd.DataFrame(
            {"win_rate_dynamic_%": [round(result_df["dyn_win_flag"].mean() * 100, 2)]},
            index=["Annual Return"]
        )
    return win_rate, result_df

# 2.4 Investment Efficiency: Dynamic vs Uniform DCA

In [ ]:
def compare_dynamic_vs_uniform_efficiency(
    file_path: str = "data/weight_price_ma20_spd_pct.csv",
    date_col: str = "date",
    price_col: str = "PriceUSD",
    dyn_weight_col: str = "weight",
    spd_pct_col: str = "spd_percentile",
    budget: float = 10_000.0,
    cheap_pct_threshold: float = 30.0,
) -> pd.DataFrame:
    """
    Compare Dynamic DCA vs Uniform DCA efficiency.

    Metrics included:
    - Total BTC Accumulation
    - Weighted Average SPD (sats per dollar)
    - Effective Average Purchase Price ($/BTC)
    - Timing Efficiency (% of capital invested in the cheapest X% of SPD)

    Required columns in the CSV:
    - date (optional but recommended)
    - price               -> BTC price
    - dynamic_weight      -> Dynamic DCA daily weight
    - spd_percentile      -> SPD percentile (0–100)
    """

    # Load and sort data
    df = pd.read_csv(file_path)

    if date_col in df.columns:
        df[date_col] = pd.to_datetime(df[date_col])
        df = df.sort_values(date_col)

    price = df[price_col].astype(float).values
    dyn_w_raw = df[dyn_weight_col].astype(float).values
    spd_pct = df[spd_pct_col].astype(float).values

    n = len(df)
    if n == 0:
        raise ValueError("DataFrame is empty.")

    # Normalize weights
    # Dynamic weights may not sum to 1 → normalize
    dyn_w = dyn_w_raw / dyn_w_raw.sum()

    # Uniform DCA: equal allocation every day
    uni_w = np.full(n, 1.0 / n)

    # Compute SPD (satoshis per dollar)
    spd = 100_000_000.0 / price  # sats per $1

    # Daily allocation & BTC purchased
    dyn_alloc = dyn_w * budget
    uni_alloc = uni_w * budget

    dyn_btc = dyn_alloc / price
    uni_btc = uni_alloc / price

    dyn_total_btc = dyn_btc.sum()
    uni_total_btc = uni_btc.sum()

    # Weighted Average SPD
    dyn_wspd = np.average(spd, weights=dyn_alloc)
    uni_wspd = np.average(spd, weights=uni_alloc)

    # Effective Average Purchase Price
    dyn_eff_price = budget / dyn_total_btc
    uni_eff_price = budget / uni_total_btc

    # Timing Efficiency
    # % of capital invested in the cheapest X% of prices (SPD percentile)
    cheap_mask = spd_pct >= cheap_pct_threshold

    dyn_timing = dyn_alloc[cheap_mask].sum() / dyn_alloc.sum()
    uni_timing = uni_alloc[cheap_mask].sum() / uni_alloc.sum()

    dyn_timing_pct = dyn_timing * 100.0
    uni_timing_pct = uni_timing * 100.0

    # Summary table
    summary = pd.DataFrame(
        {
            "Dynamic DCA": {
                "Total BTC Accumulation": round(dyn_total_btc, 1),
                "Weighted Avg SPD (sats per $)": round(dyn_wspd, 1),
                "Effective Avg Purchase Price ($/BTC)": round(dyn_eff_price, 1),
                f"Timing Efficiency (% capital in top {int(cheap_pct_threshold)}%)": round(dyn_timing_pct, 1),
            },
            "Uniform DCA": {
                "Total BTC Accumulation": round(uni_total_btc, 1),
                "Weighted Avg SPD (sats per $)": round(uni_wspd, 1),
                "Effective Avg Purchase Price ($/BTC)": round(uni_eff_price, 1),
                f"Timing Efficiency (% capital in top {int(cheap_pct_threshold)}%)": round(uni_timing_pct, 1),
            },
        }
    )

    return summary

# 3.1.1 Risk Profile Comparison: Dynamic vs Uniform DCA

In [ ]:
# Risk metric helper funcs
def sharpe_from_returns(returns: pd.Series, trading_days: int = 252) -> float:
    """Compute annualized Sharpe ratio from a daily series (here: BTC-based g_t)."""
    r = returns.dropna()
    if len(r) < 30:
        return np.nan

    mean_daily = r.mean()
    std_daily = r.std(ddof=1)
    if std_daily == 0:
        return np.nan

    return (mean_daily / std_daily) * np.sqrt(trading_days)

# Risk metric helper funcs
def volatility_from_returns(returns: pd.Series, trading_days: int = 252) -> float:
    """Compute annualized volatility from a daily series (here: BTC-based g_t)."""
    r = returns.dropna()
    if len(r) < 2:
        return np.nan
    return r.std(ddof=1) * np.sqrt(trading_days)

# Rolling 12m risk metrics by pre-cut windows=
def rolling_12m_risk_metrics_by_window(
    file_path: str = "data/port_dca.csv",
    date_col: str = "date",
    dyn_col: str = "dyn_btc",
    uni_col: str = "uni_btc",
    trading_days: int = 252,
):
    """
    Use BTC cumulative amount to compute g_t,
    then compute Sharpe / Volatility

    Assumptions:
    - port_dca.csv contains multiple 12-month windows concatenated sequentially
    - When the date is no longer strictly increasing (date[t] <= date[t-1]),
      it is treated as the start of a new window
    - For each window:
        * Use "cumulative BTC amount" to compute daily g_t:
              g_t = BTC_t - BTC_{t-1}
        * Then compute from g_t:
            - Sharpe
            - Volatility
        * Also mark whether Dynamic “beats” Uniform on each metric:
            - Sharpe: higher is better
            - Volatility: lower is better
            - Max Drawdown: higher (less negative) is better
    """
    df = pd.read_csv(file_path)
    df[date_col] = pd.to_datetime(df[date_col])

    # Detect the start of a new window when the date is no longer strictly increasing
    is_new_window = df[date_col].diff().le(pd.Timedelta(0))
    is_new_window.iloc[0] = True
    df["window_id"] = is_new_window.cumsum()

    results = []

    for wid, g in df.groupby("window_id", sort=False):
        g = g.sort_values(date_col)

        start_date = g[date_col].iloc[0]
        end_date   = g[date_col].iloc[-1]

        dyn_btc = g[dyn_col]
        uni_btc = g[uni_col]

        # g_t = BTC_t - BTC_{t-1}
        dyn_g = dyn_btc.diff()
        uni_g = uni_btc.diff()

        # Sharpe
        dyn_sharpe  = sharpe_from_returns(dyn_g, trading_days=trading_days)
        uni_sharpe  = sharpe_from_returns(uni_g, trading_days=trading_days)

        dyn_vol     = volatility_from_returns(dyn_g, trading_days=trading_days)
        uni_vol     = volatility_from_returns(uni_g, trading_days=trading_days)

        # Win/Loss logic: higher Sharpe is better
        sharpe_win   = int(dyn_sharpe  > uni_sharpe)

        # Lower volatility is better
        vol_win      = int(dyn_vol < uni_vol)

        results.append(
            {
                "window_id": wid,
                "start_date": start_date,
                "end_date": end_date,
                "dyn_sharpe": dyn_sharpe,
                "uni_sharpe": uni_sharpe,
                "dyn_volatility": dyn_vol,
                "uni_volatility": uni_vol,
                "dyn_sharpe_win": sharpe_win,
                "dyn_vol_win": vol_win,
            }
        )

    metrics_df = pd.DataFrame(results)

    # Win rate table: Dynamic DCA vs Uniform for each risk metric
    if len(metrics_df) == 0:
        win_rate_table = pd.DataFrame(
            {"dynamic dca win rate": [np.nan, np.nan]},
            index=["sharpe", "volatility_annual"],
        )
    else:
        sharpe_wr  = metrics_df["dyn_sharpe_win"].mean()
        vol_wr     = metrics_df["dyn_vol_win"].mean()

        win_rate_table = pd.DataFrame(
            {
                "win_rate_dynamic_%": [
                    round(sharpe_wr * 100, 2),
                    round(vol_wr * 100, 2),
                ]
            },
            index=["sharpe", "volatility_annual"],
        )

    return metrics_df, win_rate_table

In [ ]:
def build_portfolio_and_risk_metrics_full_history(
    file_path: str = "data/weight_price.csv",
    date_col: str = "date",
    price_col: str = "PriceUSD",
    weight_col: str = "weight",
    annual_budget: float = 10_000.0,
    start_month: int = 6,
    start_day: int = 1,
    trading_days: int = 252,
) -> tuple[pd.DataFrame, pd.DataFrame]:

    # 1. Load data
    df = pd.read_csv(file_path)
    df[date_col] = pd.to_datetime(df[date_col])
    df = df.sort_values(date_col).reset_index(drop=True)

    price = df[price_col].astype(float)

    # 2. Assign cycle year
    years = df[date_col].dt.year
    months = df[date_col].dt.month
    days = df[date_col].dt.day

    cycle_year = years.copy()
    before_cycle_start = (months < start_month) | (
        (months == start_month) & (days < start_day)
    )
    cycle_year[before_cycle_start] -= 1
    df["cycle_year"] = cycle_year

    # 3. Dynamic weights
    raw_w = df[weight_col].astype(float)
    cycle_sum = df.groupby("cycle_year")[weight_col].transform("sum")

    df["dyn_norm_weight"] = np.where(cycle_sum > 0, raw_w / cycle_sum, 0.0)
    df["dyn_invest"] = df["dyn_norm_weight"] * annual_budget

    # 4. Uniform weights
    cycle_count = df.groupby("cycle_year")["cycle_year"].transform("count")
    df["uni_norm_weight"] = np.where(cycle_count > 0, 1.0 / cycle_count, 0.0)
    df["uni_invest"] = df["uni_norm_weight"] * annual_budget

    # 5. BTC bought & held
    df["dyn_btc_bought"] = df["dyn_invest"] / price
    df["uni_btc_bought"] = df["uni_invest"] / price

    df["dyn_btc_held"] = df["dyn_btc_bought"].cumsum()
    df["uni_btc_held"] = df["uni_btc_bought"].cumsum()

    # 6. Portfolio value
    df["dyn_portfolio_value"] = df["dyn_btc_held"] * price
    df["uni_portfolio_value"] = df["uni_btc_held"] * price

    # 7. BTC-change based returns (Scheme B)
    df["dyn_g"] = df["dyn_btc_held"].diff()
    df["uni_g"] = df["uni_btc_held"].diff()

    # 8. Risk metrics
    def compute_risk_metrics_btc(btc_diff: pd.Series, portfolio_value: pd.Series):

        r = btc_diff.dropna()

        if len(r) < 30:
            return {
                "sharpe": np.nan,
                "volatility_annual": np.nan,
            }

        mean_daily = r.mean()
        std_daily = r.std(ddof=1)

        # Sharpe
        sharpe = np.nan
        if std_daily > 0:
            sharpe = (mean_daily / std_daily) * np.sqrt(trading_days)

        # Volatility
        volatility_annual = std_daily * np.sqrt(trading_days)

        return {
            "sharpe": round(sharpe, 4) if pd.notna(sharpe) else np.nan,
            "volatility_annual": round(volatility_annual, 4),
        }

    # 9. Compute both strategies
    dyn_metrics = compute_risk_metrics_btc(
        df["dyn_g"], df["dyn_portfolio_value"]
    )

    uni_metrics = compute_risk_metrics_btc(
        df["uni_g"], df["uni_portfolio_value"]
    )

    metrics_df = pd.DataFrame(
        [dyn_metrics, uni_metrics],
        index=["Dynamic DCA", "Uniform DCA"],
    )

    return df, metrics_df

# 3.1.2 Why Dynamic DCA Performs Worse on Sharpe/Volatility Metrics

In [ ]:
def mean_from_returns(returns: pd.Series, trading_days: int = 252) -> float:
    r = returns.dropna()
    if len(r) < 30:
        return np.nan
    return r.mean() * np.sqrt(trading_days)

def volatility_from_returns(returns: pd.Series, trading_days: int = 252) -> float:
    r = returns.dropna()
    if len(r) < 2:
        return np.nan
    return r.std(ddof=1) * np.sqrt(trading_days)

def rolling_12m_mean_vol_returns_by_window(
    file_path: str = "data/port_dca.csv",
    date_col: str = "date",
    dyn_col: str = "dyn_btc",
    uni_col: str = "uni_btc",
    trading_days: int = 252,
):
    df = pd.read_csv(file_path)
    df[date_col] = pd.to_datetime(df[date_col])

    # Detect the start of a new window when the date is no longer strictly increasing
    is_new_window = df[date_col].diff().le(pd.Timedelta(0))
    is_new_window.iloc[0] = True
    df["window_id"] = is_new_window.cumsum()

    results = []

    for wid, g in df.groupby("window_id", sort=False):
        g = g.sort_values(date_col)

        start_date = g[date_col].iloc[0]
        end_date   = g[date_col].iloc[-1]

        dyn_btc = g[dyn_col]
        uni_btc = g[uni_col]

        # g_t = BTC_t - BTC_{t-1}
        dyn_g = dyn_btc.diff()
        uni_g = uni_btc.diff()

        # Returns
        dyn_mean  = mean_from_returns(dyn_g, trading_days=trading_days)
        uni_mean  = mean_from_returns(uni_g, trading_days=trading_days)

        dyn_vol  = volatility_from_returns(dyn_g, trading_days=trading_days)
        uni_vol  = volatility_from_returns(uni_g, trading_days=trading_days)

        mean_win   = int(dyn_mean  > uni_mean)
        std_win   = int(dyn_vol < uni_vol)

        results.append(
            {
                "window_id": wid,
                "start_date": start_date,
                "end_date": end_date,
                "dyn_mean": dyn_mean,
                "uni_mean": uni_mean,
                "dyn_vol": dyn_vol,
                "uni_vol": uni_vol,
                "dyn_mean_win": mean_win,
                "dyn_vol_win": std_win,
            }
        )

    metrics_df = pd.DataFrame(results)

    # Win rate table: Dynamic DCA vs Uniform for each risk metric
    if len(metrics_df) == 0:
        win_rate_table = pd.DataFrame(
            {"dynamic dca win rate": [np.nan, np.nan, np.nan, np.nan]},
            index=["return_annual", "volatility_annual"],
        )
    else:
        mean_wr  = metrics_df["dyn_mean_win"].mean()
        vol_wr     = metrics_df["dyn_vol_win"].mean()

        win_rate_table = pd.DataFrame(
            {
                "win_rate_dynamic_%": [
                    round(mean_wr * 100, 2),
                    round(vol_wr * 100, 2),
                ]
            },
            index=["return_annual", "volatility_annual"],
        )

    return metrics_df, win_rate_table

In [ ]:
def plot_gt_distribution_with_tail_plotly(
    window_id: int,
    file_path: str = "data/port_dca.csv",
    date_col: str = "date",
    dyn_col: str = "dyn_btc",
    uni_col: str = "uni_btc",
    window_col: str = "window_id",
    start_col: str = "start_date",
    end_col: str = "end_date",
    bins: int = 40,
    tail_q: float = 0.9,   # Quantile threshold for defining the “tail” (e.g. 0.9 = top 10%)
):
    """
    Row 1:
      X-axis: Daily allocation g_t (in sats) = (BTC_t - BTC_{t-1}) * 1e8
      LEFT  y-axis: Frequency (Days in window)
      RIGHT y-axis: Probability (count / total days)

    Row 2:
      Shows only the right tail (x >= tail_quantile) frequency,
      so the extreme Dynamic values are more clearly visible.
    """

    df = pd.read_csv(file_path)

    # Parse dates
    if date_col in df.columns:
        df[date_col] = pd.to_datetime(df[date_col])

    for c in [start_col, end_col]:
        if c in df.columns:
            df[c] = pd.to_datetime(df[c])

    # Filter one window
    win_df = df[df[window_col] == window_id].copy()
    if win_df.empty:
        raise ValueError(f"No rows found for window_id = {window_id}")

    win_df = win_df.sort_values(date_col)

    # Title suffix
    if start_col in win_df.columns and end_col in win_df.columns:
        start_date = win_df[start_col].iloc[0].date()
        end_date = win_df[end_col].iloc[0].date()
        title_suffix = f"{start_date} → {end_date}"
    else:
        title_suffix = f"window_id = {window_id}"

    # ----- 1) Cumulative BTC → daily g_t (BTC) → convert to sats -----
    dyn_cum = win_df[dyn_col].dropna().reset_index(drop=True)
    uni_cum = win_df[uni_col].dropna().reset_index(drop=True)

    dyn_g_btc = dyn_cum.diff().dropna()
    uni_g_btc = uni_cum.diff().dropna()

    dyn_sats = dyn_g_btc * 1e8
    uni_sats = uni_g_btc * 1e8

    all_vals = np.concatenate([dyn_sats.values, uni_sats.values])

    # ----- 2) Full-range bins + probability -----
    bin_edges = np.linspace(all_vals.min(), all_vals.max(), bins)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    bin_width = bin_edges[1] - bin_edges[0]

    N_dyn = len(dyn_sats)
    N_uni = len(uni_sats)

    counts_dyn, _ = np.histogram(dyn_sats, bins=bin_edges)
    counts_uni, _ = np.histogram(uni_sats, bins=bin_edges)

    prob_dyn = counts_dyn / N_dyn
    prob_uni = counts_uni / N_uni

    dyn_mean = dyn_sats.mean()
    uni_mean = uni_sats.mean()

    # ----- 3) Define tail region -----
    tail_start = np.quantile(all_vals, tail_q)

    dyn_tail = dyn_sats[dyn_sats >= tail_start]
    uni_tail = uni_sats[uni_sats >= tail_start]

    if len(dyn_tail) == 0 and len(uni_tail) == 0:
        print("No tail values above the chosen quantile, try lowering tail_q.")
        tail_bins = 10
    else:
        tail_bins = max(6, bins // 3)

    tail_edges = np.linspace(tail_start, all_vals.max(), tail_bins)
    tail_centers = (tail_edges[:-1] + tail_edges[1:]) / 2

    counts_dyn_tail, _ = np.histogram(dyn_tail, bins=tail_edges)
    counts_uni_tail, _ = np.histogram(uni_tail, bins=tail_edges)

    # ----- 4) Plot: two-row subplot -----
    fig = make_subplots(
        rows=2,
        cols=1,
        shared_xaxes=False,
        specs=[[{"secondary_y": True}], [{"secondary_y": False}]],
        row_heights=[0.7, 0.3],
        vertical_spacing=0.12,
    )

    # ===== Row 1: Full distribution (same logic as before, now in sats) =====
    fig.add_trace(
        go.Histogram(
            x=dyn_sats,
            xbins=dict(start=bin_edges[0], end=bin_edges[-1], size=bin_width),
            name="Dynamic gₜ (count)",
            opacity=0.5,
        ),
        row=1, col=1, secondary_y=False,
    )

    fig.add_trace(
        go.Histogram(
            x=uni_sats,
            xbins=dict(start=bin_edges[0], end=bin_edges[-1], size=bin_width),
            name="Uniform gₜ (count)",
            opacity=0.5,
        ),
        row=1, col=1, secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(
            x=bin_centers,
            y=prob_dyn,
            mode="lines+markers",
            name="Dynamic prob.",
        ),
        row=1, col=1, secondary_y=True,
    )

    fig.add_trace(
        go.Scatter(
            x=bin_centers,
            y=prob_uni,
            mode="lines+markers",
            name="Uniform prob.",
        ),
        row=1, col=1, secondary_y=True,
    )

    # Mean lines (full range)
    fig.add_vline(
        x=dyn_mean,
        line_width=2,
        line_dash="dash",
        row=1, col=1,
    )

    fig.add_vline(
        x=uni_mean,
        line_width=2,
        line_dash="dash",
        row=1, col=1,
    )

    fig.add_annotation(
        x=dyn_mean,
        y=0.9,
        yref="paper",
        text="Dynamic Mean",
        showarrow=False,
        xanchor="left",
    )

    fig.add_annotation(
        x=uni_mean,
        y=0.83,
        yref="paper",
        text="Uniform Mean",
        showarrow=False,
        xanchor="left",
    )

    # ===== Row 2: Tail zoom-in =====
    fig.add_trace(
        go.Histogram(
            x=dyn_tail,
            xbins=dict(start=tail_edges[0], end=tail_edges[-1], size=tail_edges[1] - tail_edges[0]),
            name=f"Dynamic gₜ tail (>{int(tail_q*100)}% quantile)",
            opacity=0.6,
        ),
        row=2, col=1, secondary_y=False,
    )

    fig.add_trace(
        go.Histogram(
            x=uni_tail,
            xbins=dict(start=tail_edges[0], end=tail_edges[-1], size=tail_edges[1] - tail_edges[0]),
            name=f"Uniform gₜ tail (>{int(tail_q*100)}% quantile)",
            opacity=0.6,
        ),
        row=2, col=1, secondary_y=False,
    )

    fig.update_xaxes(
        title_text="Daily Allocation gₜ (sats per day)",
        row=2, col=1,
    )

    fig.update_yaxes(
        title_text="Frequency (Days in window)",
        row=1, col=1, secondary_y=False,
    )

    fig.update_yaxes(
        title_text="Probability",
        row=1, col=1, secondary_y=True,
        rangemode="tozero",
    )

    fig.update_yaxes(
        title_text=f"Tail Frequency (x ≥ {int(tail_q*100)}% quantile)",
        row=2, col=1,
    )

    fig.update_layout(
        title=(
            "Daily BTC Allocation Distribution (1-Year Window)<br>"
            f"{title_suffix} (gₜ measured in sats; bottom panel zooms in on the right tail)"
        ),
        barmode="overlay",
        legend_title_text="Strategy",
        width=950,
        height=650,
        margin=dict(l=70, r=60, t=90, b=70),
    )

    fig.show()

    # Convert the figure to a PNG and render it as notebook output (so GitHub can display it)
    png_bytes = pio.to_image(
        fig,
        format="png",
        width=950,    # Image width in pixels (increase for wider output)
        height=650,    # Image height in pixels
        scale=1         # Resolution multiplier (the most important for clarity)
    )

    # Display the PNG image inside the notebook
    display(Image(png_bytes))

# 3.2 Allocation Response to Risk  

In [ ]:
def avg_weight_after_80pct(
    file_path: str = "data/port_dca_spd.csv",
    pct_col: str = "spd_pct",
    weight_col: str = "weight",
    window_col: str = "window_id",
    threshold: float = 80.0
):
    df = pd.read_csv(file_path)
    
    # Select rows within each window where SPD percentile >= threshold (e.g., 80th)
    filtered = df[df[pct_col] >= threshold]

    # For each window, sum the weights of all days above the percentile threshold
    sum_per_window = filtered.groupby(window_col)[weight_col].sum()

    # Compute the average of these weight sums across all windows
    avg_weight = round(sum_per_window.mean(), 4)

    # Print the final result
    print("Average weight sum after 80th percentile = {} %".format(avg_weight * 100))

    return avg_weight

In [ ]:
def plot_weight_price_across_year(
    start_year: int,
    end_year: int | None = None,
    file_path: str = "data/weight_price.csv"
):
    # Load data
    df = pd.read_csv(file_path)
    df["date"] = pd.to_datetime(df["date"])
    df = df.sort_values("date")

    # If only one year is given
    if end_year is None:
        end_year = start_year + 1

    # Window: June 1 → May 31
    start_date = pd.Timestamp(f"{start_year}-06-01")
    end_date   = pd.Timestamp(f"{end_year}-05-31")

    # Filter rows
    df = df[(df["date"] >= start_date) & (df["date"] <= end_date)]

    # Plot: create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # BTC price
    fig.add_trace(
        go.Scatter(
            x=df["date"],
            y=df["PriceUSD"],
            mode="lines",
            name="BTC Price",
            line=dict(width=1.3),
            opacity=0.7
        ),
        secondary_y=False
    )

    # Weight bars (more visible)
    fig.add_trace(
        go.Bar(
            x=df["date"],
            y=df["weight"],
            name="Weight",
            marker=dict(
                color="#CC0000",     # deeper red
                opacity=0.4,           # fully solid
                line=dict(color="red", width=0.5)  # thicker outline
            ),
            width=3 * 24 * 60 * 60 * 1000  # make bars wider (3-day width in ms)
        ),
        secondary_y=True
    )

    # Add June 1 lines for each rolling window break
    for year in range(start_year, end_year + 1):
        june_1 = pd.Timestamp(f"{year}-06-01")
        if start_date <= june_1 <= end_date:
            fig.add_vline(
                x=june_1.to_pydatetime(),
                line_width=2,
                line_dash="dash",
                line_color="black"
            )
            fig.add_annotation(
                x=june_1.to_pydatetime(),
                y=1,
                yref="paper",
                text=f"{year}-06-01<br>window",
                showarrow=False,
                font=dict(size=11)
            )

    # Build yearly ticks manually (no dtick!)
    tick_vals = [pd.Timestamp(f"{y}-06-01") for y in range(start_year, end_year + 1)]
    tick_text = [f"{y}-06-01" for y in range(start_year, end_year + 1)]

    fig.update_layout(
        title=f"BTC Price and Allocation Weight ({start_year} → {end_year})",
        xaxis=dict(
            tickmode="array",
            tickvals=tick_vals,
            ticktext=tick_text
        ),
        barmode="overlay",
        width=1100,
        height=500
    )

    fig.update_yaxes(title_text="BTC Price (USD)", secondary_y=False)
    fig.update_yaxes(title_text="Weight", secondary_y=True)

    fig.show()

    # Convert the figure to a PNG and render it as notebook output (so GitHub can display it)
    png_bytes = pio.to_image(
        fig,
        format="png",
        width=1000,    # Image width in pixels (increase for wider output)
        height=500,    # Image height in pixels
        scale=2         # Resolution multiplier (the most important for clarity)
    )

    # Display the PNG image inside the notebook
    display(Image(png_bytes))


# 4.1.1 Major historical cycles with bearish and bullish phases

In [33]:
def plot_price_vs_ma200(
    file_path: str = "data/weight_price_ma200.csv",
    start_year: int = 2018,
    end_year: int = 2025,
    price_col: str = "PriceUSD",
    ma_col: str = "MA200",
    date_col: str = "date",
    buffer: float = 0.2,
):
    """
    Plot BTC historical price with MA200 (2018–2025 by default):

    - Single price line, but color-coded by regime relative to MA200:
        * Bullish   (green):   Price > MA200 * (1 + buffer)
        * Sideways  (gray):    MA200 * (1 - buffer) <= Price <= MA200 * (1 + buffer)
        * Bearish   (red):     Price < MA200 * (1 - buffer)
    - One dashed MA200 line (dark blue).
    - Also exports a PNG so GitHub can display the chart.
    """
    import pandas as pd
    import plotly.graph_objects as go
    import plotly.io as pio
    from IPython.display import Image, display

    # Load data
    df = pd.read_csv(file_path)
    df[date_col] = pd.to_datetime(df[date_col])

    # Time filter
    df = df[
        (df[date_col] >= f"{start_year}-01-01") &
        (df[date_col] <= f"{end_year}-12-31")
    ].copy()

    # --- Define regimes relative to MA200 with ±buffer ---
    upper = df[ma_col] * (1 + buffer)
    lower = df[ma_col] * (1 - buffer)

    df["regime"] = "sideways"
    df.loc[df[price_col] > upper, "regime"] = "bull"
    df.loc[df[price_col] < lower, "regime"] = "bear"

    # --- Build segments for color-changing price line ---
    segments = []
    current_regime = df["regime"].iloc[0]
    temp_rows = [df.iloc[0]]

    for i in range(1, len(df)):
        row = df.iloc[i]
        if row["regime"] != current_regime:
            segments.append((pd.DataFrame(temp_rows), current_regime))
            temp_rows = [row]
            current_regime = row["regime"]
        else:
            temp_rows.append(row)

    # last segment
    segments.append((pd.DataFrame(temp_rows), current_regime))

    fig = go.Figure()

    # Regime → color / legend name
    color_map = {
        "bull": "green",
        "sideways": "gray",
        "bear": "red",
    }
    name_map = {
        "bull": f"Price > {ma_col} × (1 + {buffer:.1f})",
        "sideways": f"|Price − {ma_col}| ≤ {int(buffer*100)}%",
        "bear": f"Price < {ma_col} × (1 − {buffer:.1f})",
    }

    # Only show each legend item once
    legend_shown = {"bull": False, "sideways": False, "bear": False}

    # --- Add colored price segments ---
    for seg_df, regime in segments:
        color = color_map[regime]
        name = name_map[regime]

        show = not legend_shown[regime]
        legend_shown[regime] = True

        fig.add_trace(
            go.Scatter(
                x=seg_df[date_col],
                y=seg_df[price_col],
                mode="lines",
                line=dict(color=color, width=2),
                name=name,
                showlegend=bool(show),
            )
        )

    # --- MA200 dashed line (more visible color) ---
    fig.add_trace(
        go.Scatter(
            x=df[date_col],
            y=df[ma_col],
            mode="lines",
            name=ma_col,
            line=dict(
                width=3,
                dash="dash",
                color="darkblue",
            ),
        )
    )

    # Layout
    fig.update_layout(
        title=f"BTC Price vs {ma_col} with Regimes (buffer = {int(buffer*100)}%) "
              f"({start_year}–{end_year})",
        xaxis_title="Date",
        yaxis_title="Price (USD)",
        width=1000,
        height=500,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0),
    )

    # Interactive output in notebook
    fig.show()

    # Convert the figure to a PNG and render it as notebook output (GitHub can display this)
    png_bytes = pio.to_image(
        fig,
        format="png",
        width=1000,   # image width in pixels
        height=500,   # image height in pixels
        scale=2,      # resolution multiplier
    )

    display(Image(png_bytes))


# 4.1.2 Duration of bull/bear regimes

In [47]:
def market_regime_frequency_two_classes(
    file_path="data/weight_price_ma200.csv",
    price_col="PriceUSD",
    ma_col="MA200",
    date_col="date",
    buffer=0.2
):
    df = pd.read_csv(file_path)
    df[date_col] = pd.to_datetime(df[date_col])

    total = len(df)

    upper = df[ma_col] * (1 + buffer)   # Bullish
    lower = df[ma_col] * (1 - buffer)   # Bearish

    df["regime"] = None
    df.loc[df[price_col] > upper, "regime"] = "bullish"
    df.loc[df[price_col] < lower, "regime"] = "bearish"

    counts = df["regime"].value_counts()

    freq_table = pd.DataFrame({
        "Regime": ["bullish", "bearish"],
        "Count": [counts.get("bullish", 0), counts.get("bearish", 0)]
    })

    freq_table["Percentage %"] = ((freq_table["Count"] / total) * 100).round(2)

    return freq_table

# 4.1.3 Dynamic DCA allocation behavior by market regime

In [ ]:
def plot_allocation_by_regime(
    file_path: str = "data/weight_price_ma200.csv",
    buffer: float = 0.2,
) -> pd.DataFrame:
    # Load data
    df = pd.read_csv(file_path)
    df["date"] = pd.to_datetime(df["date"])

    price = df["PriceUSD"]
    ma = df["MA200"]

    # Define regimes based on Price vs MA20 with a buffer
    conditions = [
        price > ma * (1 + buffer),  # clearly above MA200 → bull
        price < ma * (1 - buffer),  # clearly below MA200 → bear
    ]
    choices = ["bull", "bear"]

    df["regime"] = np.select(conditions, choices, default="sideways")

    # Mean allocation weight per regime
    regime_alloc = df.groupby("regime", as_index=False)["weight"].mean()
    regime_alloc.rename(columns={"weight": "average_daily_weight"}, inplace=True)


    # ----- Add color rule -----
    colors = regime_alloc["regime"].map(
        lambda r: "red" if r == "bear" else "steelblue"
    )

    # Plot
    plt.figure(figsize=(6, 4))
    plt.bar(regime_alloc["regime"], regime_alloc["average_daily_weight"], color=colors)
    plt.xlabel("Market Regime (based on MA200)")
    plt.ylabel("Mean Allocation Weight")
    plt.title("Mean Daily Allocation by Market Regime")
    plt.tight_layout()
    plt.show()

    return regime_alloc


# 4.2.1 Visualize MSTR’s historical buy points on the Bitcoin price chart

In [ ]:
def plot_mstr_buy_points(
    file_path: str = "data/mstr_price.csv",
    date_col: str = "date",
    price_col: str = "PriceUSD",
    mstr_btc_col: str = "mstr_btc"
):
    """
    Plot BTC price with enhanced styling,
    and mark MSTR buy points with highlighted red markers.
    """

    # Load data
    df = pd.read_csv(file_path)
    df[date_col] = pd.to_datetime(df[date_col])

    # MSTR buy days
    df_buy = df[df[mstr_btc_col] > 0]

    # Figure
    fig = go.Figure()

    # --- BTC price line ---
    fig.add_trace(
        go.Scatter(
            x=df[date_col],
            y=df[price_col],
            mode="lines",
            name="BTC Price",
            line=dict(color="steelblue", width=2.5),
            opacity=0.9,
        )
    )

    # --- MSTR buy points ---
    fig.add_trace(
        go.Scatter(
            x=df_buy[date_col],
            y=df_buy[price_col],
            mode="markers",
            name="MSTR Buy",
            marker=dict(
                color="white",
                size=10,
                line=dict(color="red", width=2),
                symbol="circle",
            )
        )
    )

    # --- Styling / decorations ---
    fig.update_layout(
        title=dict(
            text="BTC Price with MSTR Buy Points",
            x=0.5,
            font=dict(size=22)
        ),
        width=1000,
        height=500,
        template="plotly_white",
        hovermode="x unified",

        # Background
        plot_bgcolor="rgba(245, 245, 245, 1)",
        paper_bgcolor="white",

        # Axes
        xaxis=dict(
            title="Date",
            gridcolor="lightgray",
            showgrid=True,
            zeroline=False
        ),
        yaxis=dict(
            title="BTC Price (USD)",
            gridcolor="lightgray",
            showgrid=True,
            zeroline=False
        ),
        margin=dict(l=60, r=40, t=80, b=50),
    )

    fig.show()

    # Convert the figure to a PNG and render it as notebook output (so GitHub can display it)
    png_bytes = pio.to_image(
        fig,
        format="png",
        width=1000,    # Image width in pixels (increase for wider output)
        height=500,    # Image height in pixels
        scale=2         # Resolution multiplier (the most important for clarity)
    )

    # Display the PNG image inside the notebook
    display(Image(png_bytes))

# 4.2.2 Does MSTR really buy low?

In [162]:
# Load data helper
def load_mstr_price(
    file_path: str = "data/mstr_price.csv",
    date_col: str = "date",
    price_col: str = "PriceUSD",
    mstr_btc_col: str = "mstr_btc"
) -> pd.DataFrame:
    """
    Load MSTR-BTC dataset with columns:
        - date
        - PriceUSD
        - mstr_btc
    """
    df = pd.read_csv(file_path)
    df[date_col] = pd.to_datetime(df[date_col])
    df = df.sort_values(date_col).reset_index(drop=True)
    return df


# Forward-looking SPD percentile analysis
def forward_spd_percentiles(
    df: pd.DataFrame,
    date_col: str = "date",
    price_col: str = "PriceUSD",
    mstr_btc_col: str = "mstr_btc",
    horizons: list[int] = [7, 30, 90, 180],
    min_days_ratio: float = 0.8,
) -> pd.DataFrame:
    """
    For each MSTR buy event, compute the SPD percentile of the event-day price
    within the forward-looking window [t0, t0 + horizon].

    Here we use the SPD-percentile formula:

        SPD = 1 / PriceUSD   (scaled factor cancels out)
        best_SPD  = 1 / min(window_prices)   # lowest price in window
        worst_SPD = 1 / max(window_prices)   # highest price in window

        spd_pct = (your_SPD - worst_SPD) / (best_SPD - worst_SPD) * 100

    Edge handling (Option B):
        - For long horizons (e.g., 180d) near the end of the dataset,
          the forward window may be truncated.
        - We only keep the percentile if the window has at least
              min_days_ratio * horizon
          points (after dropping NaNs). Otherwise, we set it to NaN.

        Example:
            horizon = 180, min_days_ratio = 0.8
            => need at least 144 data points in [t0, t0 + 180d]
               to compute pct_180d; otherwise pct_180d = NaN.

    Returns a DataFrame with one row per event:
        event_date, price_at_event, {pct_7d, pct_30d, pct_90d, pct_180d, ...}
        where pct_*d are SPD percentiles in [0, 100].
    """
    df = df.copy().sort_values(date_col).reset_index(drop=True)

    # MSTR buy events (mstr_btc > 0)
    events = df[df[mstr_btc_col] > 0].copy()

    records = []
    for _, row in events.iterrows():
        t0 = row[date_col]
        p0 = row[price_col]

        rec = {
            "event_date": t0,
            "price_at_event": p0
        }

        # Skip if event price is missing
        if pd.isna(p0):
            for h in horizons:
                rec[f"pct_{h}d"] = np.nan
            records.append(rec)
            continue

        # SPD at event date (constant scale dropped)
        spd_0 = 1.0 / p0

        for h in horizons:
            t_end = t0 + pd.Timedelta(days=h)
            mask = (df[date_col] >= t0) & (df[date_col] <= t_end)
            window_prices = df.loc[mask, price_col].dropna()

            # Minimum number of points required for this horizon
            min_points = int(np.ceil(h * min_days_ratio))

            if len(window_prices) < min_points:
                rec[f"pct_{h}d"] = np.nan
                continue

            # Compute SPD for best/worst in the window
            p_min = window_prices.min()
            p_max = window_prices.max()

            # Guard against zero or degenerate window
            if p_min <= 0 or p_max <= 0 or p_min == p_max:
                rec[f"pct_{h}d"] = np.nan
                continue

            spd_best = 1.0 / p_min   # lowest price -> highest SPD
            spd_worst = 1.0 / p_max  # highest price -> lowest SPD

            denom = spd_best - spd_worst
            if denom == 0:
                rec[f"pct_{h}d"] = np.nan
            else:
                spd_pct = (spd_0 - spd_worst) / denom * 100.0
                rec[f"pct_{h}d"] = spd_pct

        records.append(rec)

    return pd.DataFrame(records)


def summarize_forward_percentiles(
    pct_df: pd.DataFrame,
    horizons: list[int] = [7, 30, 90, 180]
) -> pd.DataFrame:
    """
    Aggregate forward-looking SPD percentiles across all events.

    Input:
        pct_df: output of forward_price_percentiles(...)
    Output:
        DataFrame with one row per horizon:
            horizon_days, mean_percentile %, n_events
    """
    rows = []
    for h in horizons:
        col = f"pct_{h}d"
        if col not in pct_df.columns:
            continue

        vals = pct_df[col].dropna().values
        if len(vals) == 0:
            rows.append({
                "horizon_days": h,
                "mean_percentile": np.nan,
                "n_events": 0,
            })
        else:
            rows.append({
                "horizon_days": h,
                "mean_percentile": round(float(np.mean(vals)), 2),
                "n_events": len(vals),
            })

    return pd.DataFrame(rows)


In [163]:
# Random timing comparison (using forward-looking SPD percentiles)
def random_timing_forward_percentiles(
    df: pd.DataFrame,
    date_col: str = "date",
    price_col: str = "PriceUSD",
    mstr_btc_col: str = "mstr_btc",
    horizons: list[int] = [7, 30, 90, 180],
    n_iter: int = 1000,
    random_state: int | None = 42,
    min_days_ratio: float = 0.8,
) -> pd.DataFrame:
    """
    Monte Carlo random timing test (SPD-percentile-based).

    Idea:
        - We want to know: if an investor picks buy dates at random
          (same number of buys as MSTR), how good are their entry
          prices compared to future prices in terms of SPD percentile.

    For each iteration:
        1. Randomly pick N dates (N = # of MSTR buy events),
           from all dates with non-missing prices.
        2. For each sampled "event" date t0 and each horizon h:
               - Consider the forward window [t0, t0 + h]
               - Drop NaN prices
               - Require at least min_days_ratio * h points, otherwise skip
               - Compute SPD-based percentile using:

                     SPD = 1 / price
                     best_SPD  = 1 / min(window_prices)   # lowest price
                     worst_SPD = 1 / max(window_prices)   # highest price

                     spd_pct = (SPD_t0 - worst_SPD) / (best_SPD - worst_SPD) * 100

        3. For each horizon, store the average SPD percentile across all
           valid sampled events in this iteration.

    Returns:
        A DataFrame of shape [n_iter x len(horizons)] with columns:
            mean_pct_7d, mean_pct_30d, mean_pct_90d, mean_pct_180d, ...
        Each row = ONE random-timing experiment.
        Each value = the average forward-looking SPD percentile for that experiment.
    """
    rng = np.random.default_rng(random_state)

    # Sort by date to ensure proper time ordering
    base = df.copy().sort_values(date_col).reset_index(drop=True)

    # Real MSTR events: only used to match number of buys (N)
    real_events = base[base[mstr_btc_col] > 0][date_col].values
    n_events = len(real_events)
    if n_events == 0:
        raise ValueError("No MSTR buy events (mstr_btc > 0) found.")

    # Candidate dates: all dates with non-missing prices
    candidate_df = base.dropna(subset=[price_col]).reset_index(drop=True)
    candidate_dates = candidate_df[date_col].values

    if len(candidate_dates) < n_events:
        raise ValueError("Not enough candidate dates to sample random events.")

    # Use date as index for fast lookup / slicing
    df_idx = base.set_index(date_col)

    records: list[dict[str, float]] = []

    for _ in range(n_iter):
        # Sample N random "buy dates" (no replacement)
        sampled_dates = rng.choice(candidate_dates, size=n_events, replace=False)
        horizon_means: dict[str, float] = {}

        for h in horizons:
            pcts: list[float] = []
            min_points = int(np.ceil(h * min_days_ratio))

            for t0 in sampled_dates:
                t0 = pd.Timestamp(t0)
                t_end = t0 + pd.Timedelta(days=h)

                if t0 not in df_idx.index:
                    continue

                # Forward window [t0, t0 + h]
                window_prices = df_idx.loc[
                    (df_idx.index >= t0) & (df_idx.index <= t_end),
                    price_col
                ].dropna()

                # window too short => skip this event for this horizon
                if len(window_prices) < min_points:
                    continue

                p0 = df_idx.loc[t0, price_col]
                if pd.isna(p0) or p0 <= 0:
                    continue

                # SPD at event date
                spd_0 = 1.0 / p0

                # Best & worst SPD in the window
                p_min = window_prices.min()
                p_max = window_prices.max()
                if p_min <= 0 or p_max <= 0 or p_min == p_max:
                    continue

                spd_best = 1.0 / p_min   # lowest price -> highest SPD
                spd_worst = 1.0 / p_max  # highest price -> lowest SPD

                denom = spd_best - spd_worst
                if denom == 0:
                    continue

                spd_pct = (spd_0 - spd_worst) / denom * 100.0
                pcts.append(spd_pct)

            # Average SPD percentile across all valid sampled events (this iteration)
            if len(pcts) == 0:
                horizon_means[f"mean_pct_{h}d"] = np.nan
            else:
                horizon_means[f"mean_pct_{h}d"] = float(np.mean(pcts))

        records.append(horizon_means)

    return pd.DataFrame(records)


In [ ]:
def plot_percentile_distributions(
    rand_df: pd.DataFrame,
    mstr_summary: pd.DataFrame,
    horizons: list[int] = [7, 30, 90, 180]
):
    """
    Draw distribution plots of random timing percentiles
    and overlay MSTR's average percentile as a vertical line.
    """

    # 2x2 subplot
    fig = sp.make_subplots(
        rows=2, cols=2,
        subplot_titles=[f"{h}-Day Forward Percentile" for h in horizons]
    )

    positions = [(1,1), (1,2), (2,1), (2,2)]

    for idx, (h, pos) in enumerate(zip(horizons, positions), start=1):
        row, col = pos
        colname = f"mean_pct_{h}d"

        # histogram
        fig.add_trace(
            go.Histogram(
                x=rand_df[colname],
                nbinsx=40,
                opacity=0.75
            ),
            row=row, col=col
        )

        # MSTR value
        mstr_val = mstr_summary.loc[mstr_summary["horizon_days"] == h, "mean_percentile"].iloc[0]

        # vertical line
        fig.add_vline(
            x=mstr_val,
            line_color="red",
            line_width=3,
            line_dash="dash",
            row=row, col=col
        )

        fig.add_annotation(
            x=mstr_val,
            xref=f"x{idx}",
            yref=f"y{idx}",
            text=f"MSTR = {mstr_val:.3f}",
            showarrow=False,
            font=dict(color="red", size=12),
            yanchor="bottom",
        )

    fig.update_layout(
        height=500,
        width=700,
        title="Forward SPD Percentile Distribution vs MSTR Percentile",
        showlegend=False
    )

    fig.show()

    # Convert the figure to a PNG and render it as notebook output (so GitHub can display it)
    png_bytes = pio.to_image(
        fig,
        format="png",
        height=500,
        width=700,
        scale=1
    )

    # Display the PNG image inside the notebook
    display(Image(png_bytes))


In [ ]:
def percentile_ttest_mstr_rand(
    rand_df, mstr_summary, horizons=[7, 30, 90, 180]
):
    rows = []

    for h in horizons:
        col = f"mean_pct_{h}d"
        rand_vals = rand_df[col].dropna().values
        
        rand_mean = rand_vals.mean()
        rand_std = rand_vals.std(ddof=1)

        mstr_mean = mstr_summary.loc[
            mstr_summary.horizon_days == h,
            "mean_percentile"
        ].iloc[0]

        # t = (MSTR - Random) / std_random
        t_stat = (mstr_mean - rand_mean) / rand_std

        # one-sided test: H1: mstr_mean < rand_mean  (left-side)
        df = len(rand_vals) - 1
        p_value = t.cdf(t_stat, df=df)   # left-tail

        rows.append({
            "horizon_days": h,
            "mstr_mean": mstr_mean,
            "rand_mean": rand_mean,
            "std_rand": rand_std,
            "t_stat": t_stat,
            "p_value_one_sided": p_value
        })

    return pd.DataFrame(rows)
